In [1]:
!pip install graphdatascience 

  Using cached graphdatascience-1.10-py3-none-any.whl.metadata (7.7 kB)
  Using cached multimethod-1.12-py3-none-any.whl.metadata (9.6 kB)
  Using cached neo4j-5.22.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached textdistance-4.6.2-py3-none-any.whl.metadata (18 kB)
Using cached graphdatascience-1.10-py3-none-any.whl (1.6 MB)
Using cached multimethod-1.12-py3-none-any.whl (10 kB)
Using cached neo4j-5.22.0-py3-none-any.whl (293 kB)
Using cached textdistance-4.6.2-py3-none-any.whl (31 kB)


In [2]:
!pip install python-Levenshtein

  Using cached python_Levenshtein-0.25.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached Levenshtein-0.25.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.3 kB)
  Using cached rapidfuzz-3.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached python_Levenshtein-0.25.1-py3-none-any.whl (9.4 kB)
Using cached Levenshtein-0.25.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (177 kB)
Using cached rapidfuzz-3.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.4 MB)


In [3]:
from graphdatascience import GraphDataScience
from getpass import getpass
import pandas as pd
import numpy as np

In [7]:
!python main.py --dataset emails --nt 1000 --nq 1000 --epochs 20 --save-split --recall --save-embed --save-model

2024-07-07 18:54:29.163901: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
# Loading time: 0.6244535446166992
# loading indices from file
# Unique signature     : 39
# Maximum length       : 44
# Sampled Train Items  : 1000
# Sampled Query Items  : 1000
# Number of Base Items : 93905
# Number of Items      : 95905
# loading dist and knn from file
# train dist : (1000, 1000)
# query dist : (1000, 93905)
/opt/conda/lib/python3.10/site-packages/torch/u

In [4]:
email_df = pd.read_csv("data/emails", header=None)
email_df.columns=['email']

In [5]:
email_df.head()

,email
0,rhonda82@yahoo.com
1,mvelazquez@hotmail.com
2,sandrawilliams@chavez-morales.com
3,caleb08@pacheco.com
4,esmith@chen.com


In [15]:
!ls model/808/cnn/emails/nt1000_nq1000

base_idx.npy  query_dist.npy  query_knn.npy   train_idx.npy
model.torch   query_idx.npy   train_dist.npy  train_knn.npy


In [8]:
base_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/embedding_xb.npy')
train_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/embedding_xt.npy')
query_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/embedding_xq.npy')

In [9]:
embedding_df = pd.concat([pd.DataFrame(base_embedding), pd.DataFrame(train_embedding), pd.DataFrame(query_embedding)])

In [10]:
base_index = train_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/base_idx.npy')
train_index = train_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/train_idx.npy')
query_index = train_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/query_idx.npy')

In [11]:
embedding_index = pd.concat([pd.Series(base_index), pd.Series(train_index), pd.Series(query_index)])

In [12]:
embedding_df.index = embedding_index

In [13]:
embedding_df['embedding'] = embedding_df.apply(lambda row: row.tolist(), axis=1)

In [14]:
embedding_df = embedding_df['embedding'].copy()

In [15]:
embedding_df = email_df.merge(embedding_df, left_index=True, right_index=True)

In [16]:
embedding_df.head()

,email,embedding
0,rhonda82@yahoo.com,"[-0.03655370697379112, -0.04969412460923195, -..."
1,mvelazquez@hotmail.com,"[-0.02047136053442955, -0.04047189652919769, -..."
2,sandrawilliams@chavez-morales.com,"[-0.04562493786215782, -0.08655732125043869, 0..."
3,caleb08@pacheco.com,"[0.0035057964269071817, -0.027541927993297577,..."
4,esmith@chen.com,"[0.0009165378869511187, -0.04006942734122276, ..."


In [19]:
embedding_df.loc[base_index, 'Split'] = 'Base'
embedding_df.loc[train_index, 'Split'] = 'Train'
embedding_df.loc[query_index, 'Split'] = 'Query'

In [22]:
neo4j_password = getpass("neo4j password")

neo4j password ········


In [23]:
gds = GraphDataScience("neo4j+s://3bddbcd7.databases.neo4j.io", auth=("neo4j", neo4j_password))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('advertisedListenAddress' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('serverLocation' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'


In [24]:
gds.run_cypher("""CREATE CONSTRAINT email_node_key FOR (e:Email) REQUIRE e.address IS NODE KEY""")

""


In [25]:
gds.run_cypher("""
UNWIND $data AS row
CALL {
    WITH row
    MERGE (e:Email {address:row['email']})
    SET e.split = row['Split']
    WITH row, e
    CALL db.create.setNodeVectorProperty(e, 'editEmbedding', row['embedding']) 
} IN CONCURRENT TRANSACTIONS OF 10000 rows""",
              {"data": embedding_df.to_dict("records")})

""
